# Importing packages

In [1]:
import requests
#from keras.applications import ResNet50
#from keras.applications.resnet import decode_predictions, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import sys
from dotenv import load_dotenv
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from helpers import *

# Load environment variables from the secret.env file
load_dotenv()

# Access environment variables using os.getenv() or os.environ
api_key = os.getenv("API_KEY")
api_url = os.getenv("API_URL")

2024-12-21 17:19:54.243714: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 17:19:54.244124: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-21 17:19:54.247002: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-21 17:19:54.253113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734797994.264312   21001 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734797994.26

In [2]:
# We can check which models ResNet Models are available in keras
available_models = [name for name in dir(tf.keras.applications) if 'resnet' in name.lower()]
print(available_models)

['InceptionResNetV2', 'ResNet101', 'ResNet101V2', 'ResNet152', 'ResNet152V2', 'ResNet50', 'ResNet50V2', 'inception_resnet_v2', 'resnet', 'resnet50', 'resnet_v2']


# <strong>Step 1:</strong> Get a trainable RestNet model

### Parameter settings for trainable model compilation
- We set the model name to be used
- Can be 'ResNet101', 'ResNet101V2', 'ResNet152', 'ResNet152V2', 'ResNet50', 'ResNet50V2'
- Run the cell to obtained a ResNet model ready to be trained

In [3]:
# Uncomment the model_name you want to use

#model_name = "ResNet50" 
#model_name = "ResNet50V2" 
#model_name = "ResNet101" 
#model_name = "ResNet101V2" 
#model_name = "ResNet152" 
model_name = "ResNet152V2" 

# We load the model
model = compile_new_model(model_name)

# We display the model summary (optional)
# model.summary()

Model 'ResNet152V2' found in tf.keras.applications.


2024-12-21 17:19:56.614265: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Base_model 'ResNet152V2' loaded successfully.
New ResNet152V2 compiled successfully and is ready to be trained!


# <strong>Step 2:</strong> Build the training and validation dataset

### Parameter settings for dataset collection
- Set the start_date using "YYYY-MM-DD" format (ex: "2020-08-01")
- Set the end_date using "YYYY-MM-DD" format (ex: "2020-08-01")
- Set the tags as string or list (ex: tags = ['vine', 'grass', 'ground'] or tags = 'ground') 
- Run the cell to obtained the training and validation dataset 

In [4]:
# We set the start date and end date for the training data
start_date = "2020-09-01"
end_date = "2021-01-01"

# We set the tag (i.e. 'vine', 'grass' or 'ground') or the list of tags we want to train the model on (i.e ['vine', 'grass', 'ground'])
tags = ['vine', 'grass', 'ground']

# We collect the image urls for the selected tags and dates through the API
image_urls = get_image_urls_with_multiple_tags(tags, start_date, end_date, api_key, api_url)

Number of urls collected for vine: 6
Number of urls collected for grass: 1
Number of urls collected for ground: 1


In [5]:
train_dataset, val_dataset = make_train_test_dataset_from_image_urls(model_name, tags, start_date, end_date, api_key, api_url)                                     

Number of urls collected for vine: 6
Number of urls collected for grass: 1
Number of urls collected for ground: 1
Number of images found: 8
Image Signature=03b0dc2e9d736b23918579a8fd214ab442a5cdcc downloaded successfully
Image Signature=1d3347deb0808431ed35c3c0b63898888e128b1f downloaded successfully
Image Signature=4f4759f79383a200d6d1a37112abfd932b2f38a3 downloaded successfully
Image Signature=c8c7cb72feeca76f3c402d9eb74252b79a50255a downloaded successfully
Image Signature=56c7f1f568f954a761f0d47021fe050deb72b236 downloaded successfully
Image Signature=954b9de4fd9ff6d4a4580bd73c220dc6d623953a downloaded successfully
Image Signature=dceb1ba7d0b6124b251534bfb895a270fa8114c8 downloaded successfully
Image Signature=421a5e9860b37ed225d36cc263feb736bea39172 downloaded successfully
Preprocess_input and encode_predictions function for 'ResNet152V2' loaded successfully.
Training and validation datasets created successfully!


# <strong>Step 3:</strong> We train the model

- Set the number of epoch  
- Run the cell to train the model 

In [9]:
# Train the model
number_of_epochs = 1
history = model.fit(train_dataset, validation_data=val_dataset, epochs=number_of_epochs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6667 - loss: 2.5545 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


# <strong>Step 4:</strong> We save the trained model

- Run the cell to save the model 

In [17]:
# Save the trained model
model.save(f"model_archives/{model_name}_trained_model_{datetime.now().strftime("%Y%m%d-%H%M%S")}.keras")

In [ ]:
# Load the trained model
loaded_model = tf.keras.models.load_model("path_to_directory/resnet_trained_model")

# Verify it works
loaded_model.summary()

In [ ]:
# Define a checkpoint directory
checkpoint_dir = "path_to_directory/checkpoints"
checkpoint_prefix = f"{checkpoint_dir}/ckpt"

# Create a callback to save model checkpoints
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,  # Saves only weights, not the full model
    save_best_only=True,     # Saves the best model (based on validation loss)
    monitor="val_loss",      # Metric to monitor
    verbose=1
)

# Train the model with the checkpoint callback
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[checkpoint_callback]
)
